In [12]:
import os
import glob
import sys
import pandas as pd
import numpy as np
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Dataset')  # Adjust the path based on the relative location
from utils import *


def get_files_recursive(folder_path, extension):
    # Get all .off files from the folder and its subfolders
    files = glob.glob(os.path.join(folder_path, "**", "*."+extension), recursive=True)
    
    # Filter out files that contain 'SIMPL' in their name
    #files = [file for file in files if 'SMPL' not in os.path.basename(file)]
    
    return files


def get_files_recursive_mot(folder_path, mot):
    # Get all .off files from the folder and its subfolders
    files = glob.glob(os.path.join(folder_path, "**", "*"+mot+"*"), recursive=True)
    
    # Filter out files that contain 'SIMPL' in their name
    #files = [file for file in files if 'SMPL' not in os.path.basename(file)]
    
    return files

def read_paths_from_txt(txt_file):
    """Reads a list of file paths from a .txt file."""
    with open(txt_file, 'r') as f:
        paths = f.readlines()
    # Strip any whitespace characters like `\n` at the end of each line
    paths = [path.strip() for path in paths]
    return paths

###  Répartition des axes de sym sur TOUT modelenet40

In [45]:
occurences = {}
paths_list_axe = get_files_recursive_mot("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths", "and_axe")

for path_c in paths_list_axe:
    cat = os.path.basename(path_c).split('_meshes')[0]
    occurences[cat] = {}
    couple_p_a = read_paths_from_txt(path_c)
    occurences[cat]['nb_objet_total'] = len(couple_p_a)
    for k in range(0, 5):
        occurences[cat][str(k)] = 0
    occurences[cat]['+'] = 0

    for c in couple_p_a:
        nb_axe = c.split(',')[-1]
        occurences[cat][nb_axe]+=1       
        
list_cat = list(occurences.keys())
list_axe = list(occurences[list_cat[0]].keys())
df_occurences = pd.DataFrame(columns=['categorie']+list(occurences[list_cat[0]].keys()))
for i in range(len(list_cat)):
    df_occurences.loc[i+1] = [list_cat[i]]+[occurences[list_cat[i]][k] for k in list_axe]
    
df_occurences.loc[len(list_cat)+1] = ['Total']+list(np.sum(np.array(df_occurences)[:,1:], axis=0)) 

#df_occurences.to_csv('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/csv_files/All/repartion_axe_modelent40.csv', index=False)

df_occurences

,categorie,nb_objet_total,0,1,2,3,4,+
1,night_stand,286,25,234,5,0,22,0
2,cup,99,3,40,0,0,0,56
3,airplane,726,101,625,0,0,0,0
4,tv_stand,367,80,237,46,0,4,0
5,vase,576,77,9,52,0,17,421
6,bench,193,7,102,83,0,1,0
7,car,296,111,185,0,0,0,0
8,dresser,290,21,269,0,0,0,0
9,chair,0,0,0,0,0,0,0
10,Total,2833,425,1701,186,0,44,477


### Repartition BVS dispo et axe de symetrie

In [14]:
# Nb bvs dispo/cat
paths_bvs_dispo = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_SMPLER_files_ModelNet40_remeshing_iso_user-study-844-ok.txt")
dict_bvs_dispo = {cat : int(np.sum([1 for i in range(len(paths_bvs_dispo)) if cat in paths_bvs_dispo[i]])) for cat in set(paths_bvs_dispo[i].split('/')[0] for i in range(len(paths_bvs_dispo)))}
# BVS dispo en foncton des axe de sym
dict_bvs_dispo_axe_sym = {}
for cat_m in set(paths_bvs_dispo[i].split('/')[0] for i in range(len(paths_bvs_dispo))):
    paths_bvs_cat = [paths_bvs_dispo[i] for i in range(len(paths_bvs_dispo)) if cat_m in paths_bvs_dispo[i]]
    paths_and_axes_sym = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/"+cat_m+"_meshes_and_axe.txt")
    dict_bvs_dispo_axe_sym[cat_m] = {'0':[], '1':[], '2':[], '3': [], '4': [], '+': []}
    for path in paths_and_axes_sym:
        num_path = path.split('_SMPLER')[0].split(cat_m+"_")[-1]
        # si on a le fichier bvs 
        if int(np.sum([num_path in paths_bvs_cat[u] for u in range(len(paths_bvs_cat))])):
            path_obj = path.split(',')[0].replace('Dataset-aligned/', '').replace("_aligned_ok_US.obj","")
            if ',1' in path: dict_bvs_dispo_axe_sym[cat_m]['1'].append(path_obj)
            elif ',2' in path: dict_bvs_dispo_axe_sym[cat_m]['2'].append(path_obj)
            elif ',3' in path: dict_bvs_dispo_axe_sym[cat_m]['3'].append(path_obj)
            elif ',4' in path: dict_bvs_dispo_axe_sym[cat_m]['4'].append(path_obj)
            elif ',0' in path: dict_bvs_dispo_axe_sym[cat_m]['0'].append(path_obj)
            else: dict_bvs_dispo_axe_sym[cat_m]['+'].append(path_obj)

In [43]:
df_repartion_axe_et_bvs = pd.DataFrame(columns=['categorie']+list(occurences[list_cat[0]].keys())+[" --- ", "BVS dispo"]+list(occurences[list_cat[0]].keys())[1:])

for i in range(len(list_cat)):
    a= [list_cat[i]]+[occurences[list_cat[i]][k] for k in list_axe]+ [" --- ", dict_bvs_dispo[list_cat[i]]]+[len(dict_bvs_dispo_axe_sym[list_cat[i]][k]) for k in list_axe[1:]]
    df_repartion_axe_et_bvs.loc[i+1] = a
    
df_repartion_axe_et_bvs.loc[len(list_cat)+1] = ['Total']+list(np.sum(np.array(df_repartion_axe_et_bvs)[:,1:8], axis=0)) + [" --- "]+ list(np.sum(np.array(df_repartion_axe_et_bvs)[:,9:], axis=0))
    
#df_repartion_axe_et_bvs.to_csv('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/csv_files/All/repartion_axe_et_bvs_modelent40.csv', index=False)

In [44]:
df_repartion_axe_et_bvs

,categorie,nb_objet_total,0,1,2,3,4,+,---,BVS dispo,0,1,2,3,4,+
1,night_stand,286,25,234,5,0,22,0,---,40,1,31,2,0,6,0
2,cup,99,3,40,0,0,0,56,---,22,1,10,0,0,0,11
3,airplane,726,101,625,0,0,0,0,---,185,20,165,0,0,0,0
4,tv_stand,367,80,237,46,0,4,0,---,63,10,39,14,0,0,0
5,vase,576,77,9,52,0,17,421,---,157,18,3,13,0,5,118
6,bench,193,7,102,83,0,1,0,---,47,1,27,19,0,0,0
7,car,296,111,185,0,0,0,0,---,38,9,29,0,0,0,0
8,dresser,290,21,269,0,0,0,0,---,43,3,40,0,0,0,0
9,chair,0,0,0,0,0,0,0,---,249,0,0,0,0,0,0
10,Total,2833,425,1701,186,0,44,477,---,844,63,344,48,0,11,129
